In [5]:
import os
from glob import glob
import json
from django.conf import settings
from plotly import graph_objects as go
from dados import dbdata
import dash
import dash_core_components as dcc
import dash_html_components as html

#CHAMA OS PARAMETROS DE NOTIF
notif = dbdata.NotificationResume()
notif.__dir__()

###
df_rj = notif.get_cities_alert_by_state(
    state_name='Rio de Janeiro',
    disease='dengue',
    epi_year_week=201805,
)
df_rj.municipio_geocodigo = df_rj.municipio_geocodigo.apply(str)

###
# FAZ A BUSCA DE TODOS OS JSONS DISPONIVEIS

geojson_rj_path = os.path.join(
    settings.STATIC_ROOT,
    'geojson_simplified',
    'RJ.json'
)

with open(geojson_rj_path, 'r') as f:
    geojson_rj = json.load(f)
    
# CARREGA OS DADOS DE PROPERTIES PARA AS VARIAVEIS EM STRING

for features in geojson_rj['features']:
        data_pop = str(features['properties']['populacao'])
        

for features in geojson_rj['features']:
        data_nome = str(features['properties']['nome'])


In [6]:
import plotly.graph_objects as go
import pandas as pd


data_text = "{} : {} Habitantes.".format(data_nome, data_pop)
colors = ['rgb(0,255,0)', 'rgb(255,255,0)', 'rgb(255,128,0)', 'rgb(255,0,0)']
legend_alert = {6:'Alerta Verde', 7:'Alerta Amarelo', 8:'Alerta Laranja', 9:'Alerta Vermelho'}


fig = go.Figure()

# scatter chart for legend
for i in range(6,10)[::-1]:    
    fig.add_trace(go.Scattermapbox(
        lon = ['-43.176'],
        lat = ['-22.187'],
        marker_opacity=0.5,
        mode='lines',
        name = legend_alert[i],
        marker=
            dict(
                size=15,
                color=colors[i-6])
        )
    )

fig.add_trace(
    go.Choroplethmapbox(
        geojson=geojson_rj, 
        locations=df_rj.municipio_geocodigo, 
        z=df_rj.level_alert,
        text=data_text,
        hoverinfo="text",
        colorscale = colors,
        marker_opacity=0.5,
        autocolorscale = False,
        showscale = False,
    ))

fig.update_layout(
    mapbox_pitch=45,
    mapbox_style="carto-positron",
    mapbox_zoom=6.5, 
    mapbox_center = {"lat": -22.476, "lon": -43.127},
    updatemenus=[
    dict(
        buttons=([
            dict(
                args=[
                    {
                    "mapbox.zoom": "6.5",
                    "mapbox.center.lon": "-43.127",
                    "mapbox.center.lat": "-22.476",
                    "mapbox.bearing": "0",
                    }
                ],
                label="Reset Zoom",
                method="relayout",
            )
        ]),
        direction="left",
        pad={"r": 0, "t": 0, "b": 0, "l": 0},
        showactive=False,
        type="buttons",
        x=0.82,
        y=0.02,
        xanchor="left",
        yanchor="bottom",
        bgcolor="grey",
        borderwidth=1,
        bordercolor="#6d6d6d",
        font=dict(color="#FFFFFF"),
    )])



fig.show()

In [21]:
import os
from glob import glob
import json
from django.conf import settings
from plotly import graph_objects as go
from dados import dbdata

dbdata.__dir__()

series = dbdata.load_serie_cities()

TypeError: load_serie_cities() missing 1 required positional argument: 'geocodigos'

In [11]:
# Load data https://info.dengue.mat.br/...
df_gender = pd.read_csv(
    'http://127.0.0.1:8000/api/notif_reduced?'
    'chart_type=age_gender&diseases=Dengue&state_abv=CE'
)
df_diseases = pd.read_csv(
    'http://127.0.0.1:8000/api/notif_reduced?'
    'chart_type=disease&diseases=Dengue&state_abv=CE'
)

all_options = {
    'Disease': ['Chikungunya', u'Dengue', 'Zika'],
    'Gender': ['Homens', u'Mulheres'],
}

data_gender = {
    'Homens': {'x': df_diseases.category, 'type': 'bar', 'marker': {'color':"red"}, 'y': df_gender.Homem},
    'Mulheres': {'x': df_diseases.category,'type': 'bar', 'marker': {'color':"blue"}, 'y': df_gender.Mulher},
}

data_age = {
    'Homens': {'x' : df_gender.category,  'y': df_gender.Homem, 'type': 'bar', 'marker': {'color':"red"}, 'name':'Homem'},
    'Mulheres':{'x' : df_gender.category, 'y': df_gender.Mulher, 'type': 'bar', 'marker': {'color':"blue"}, 'name':'Mulher'}

}

URLError: <urlopen error [Errno 111] Connection refused>

In [12]:
from django_plotly_dash import DjangoDash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash()


app.layout = html.Div(
    html.Div([
        html.Div([
            html.P('Selecione o filtro:'),
            dcc.Checklist(
                id='Disease',
                value=['Homens'],
                style={
                    'borderBottom': 'thin lightgrey solid',
                    'backgroundColor': 'rgb(250, 250, 250)',
                    'padding': '0px 0px'},
                labelStyle={'display': 'inline-block'}
            )
            ],className='six columns'),
        html.Div([
            dcc.RadioItems(
                id='Gender',
                options=[
                    {'label': k, 'value': k}
                    for k in all_options.keys()
                ],
                value='All',
                # labelStyle={'displa': 'inline-block'},
                style={
                    'borderBottom': 'thin lightgrey solid',
                    'backgroundColor': 'rgb(250, 250, 250)',
                    'padding': '5px 0px'},
                labelStyle={'display': 'inline-block'})
                ],className="row"),
        html.Div([
            html.Div([
                    dcc.Graph(
                        id='graph-gender',
                        style={
                           'padding': '0px 0px 0px 0px',#Distancia entre graficos
                           "height": "300px",
                            "width": "100%",
                        },
                    ),
                ])
            ],className='six columns'),
            html.Div([
                    dcc.Graph(
                        id='graph-age',
                        style={
                           'padding': '0px 0px 0px 0px',
                           "height": "300px",
                            "width": "100%",
                        },
                    ),
                ],className="six columns")
                ],className="row")
        )

@app.callback(
    dash.dependencies.Output('Disease', 'options'),
    [dash.dependencies.Input('Gender', 'value')],
    # [dash.dependencies.Input('Gender', 'value')],
)
def set_gender_options(selected_gender):
    return [
        {'label': i, 'value': i}
        for i in all_options[selected_gender]
    ]

@app.callback(
    dash.dependencies.Output('graph-gender', 'figure'),
    [dash.dependencies.Input('Disease', 'value')],
)
def update_graph_gender(selector):
    data = []
    for value in selector:
        data.append(
            {
                'x': data_gender[value]['x'],
                'y': data_gender[value]['y'],
                'type': 'bar',
                'name': value,
            }
        )
    figure = {
        'data': data,
        "layout": go.Layout(
            showlegend=True,
            yaxis={"title": "Casos"},
            xaxis={"title": "Distribuição por Gênero", "tickangle": 0},
            #margin={'l': 555, 'b': 70, 't': 50, 'r': 5},
            font=dict(family='sans-serif', size=12, color='#000'),
        ),
    }
    return figure


@app.callback(
    dash.dependencies.Output('graph-age', 'figure'),
    [dash.dependencies.Input('Disease', 'value')],
)
def update_graph_age(selector):
    data = []
    for value in selector:
        data.append(
            {
                'x': data_age[value]['x'],
                'y': data_age[value]['y'],
                'type': 'bar',
                'name': value,
            }
        )
    figure = {
        'data': data,
        "layout": go.Layout(
            showlegend=False,
            yaxis={"title": "Casos"},
            xaxis={"title": "Distribuição por Idade", "tickangle": 45},
            barmode='stack',
            #margin={'l': 275, 'b': 70, 't': 5, 'r': 275},
            font=dict(family='sans-serif', size=12, color='#000'),
        ),
    }
    return figure

NameError: name 'all_options' is not defined